# 1. Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D, Flatten, Layer
from tensorflow.keras.layers import Input, Attention, Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings("ignore")

# 2. Data Cleaning

In [2]:
df = pd.read_csv('/kaggle/input/hate-speech/Hate speech excel file (1).csv', encoding='latin-1')

In [3]:
df.head(10)

,Unnamed: 0,id,segment_id,title,caption_segment,sentiment
0,0,#NAME?,#NAME?,LIVE l PM Stresses On Zero Tolerance To Terror...,freedom and civilization it knows no boundarie...,1.0
1,1,lpYVmd8a1rY,lpYVmd8a1rY_1,Reading park stabbings: UK Labour leader react...,well this was just a horrific incident and our...,1.0
2,2,lpYVmd8a1rY,lpYVmd8a1rY_2,Reading park stabbings: UK Labour leader react...,runs its course but I will want to work with t...,-1.0
3,3,lpYVmd8a1rY,lpYVmd8a1rY_3,Reading park stabbings: UK Labour leader react...,moment investigation has to take its course bu...,1.0
4,4,lpYVmd8a1rY,lpYVmd8a1rY_4,Reading park stabbings: UK Labour leader react...,our responders the NHS and the police yet agai...,-1.0
5,5,qOl5ox674vk,qOl5ox674vk_1,JUST IN: DECLARE NSCN-IM AS TERRORIST OUTFIT: ...,[Music] hello and welcome to impact news this ...,1.0
6,6,qOl5ox674vk,qOl5ox674vk_2,JUST IN: DECLARE NSCN-IM AS TERRORIST OUTFIT: ...,identity protection council t.h manihar said t...,1.0
7,7,qOl5ox674vk,qOl5ox674vk_3,JUST IN: DECLARE NSCN-IM AS TERRORIST OUTFIT: ...,but to book the demanding unconditional releas...,1.0
8,8,1kaR3I2M0es,1kaR3I2M0es_1,Russian Imperial Movement designated as terror...,today the State Department is designating the ...,1.0
9,9,1kaR3I2M0es,1kaR3I2M0es_2,Russian Imperial Movement designated as terror...,plays a prominent role in trying to rally like...,1.0


In [4]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]', '') 
    return text

In [5]:
df['cleaned_text'] = df['caption_segment'].apply(preprocess_text)

In [6]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])
padded_sequences = pad_sequences(sequences, maxlen=115, padding='post')
max_len = 115
vocab_size = len(tokenizer.word_index) + 1

In [7]:
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 1.0 else 0)

In [8]:
df.head()

,Unnamed: 0,id,segment_id,title,caption_segment,sentiment,cleaned_text
0,0,#NAME?,#NAME?,LIVE l PM Stresses On Zero Tolerance To Terror...,freedom and civilization it knows no boundarie...,1,freedom and civilization it knows no boundarie...
1,1,lpYVmd8a1rY,lpYVmd8a1rY_1,Reading park stabbings: UK Labour leader react...,well this was just a horrific incident and our...,1,well this was just a horrific incident and our...
2,2,lpYVmd8a1rY,lpYVmd8a1rY_2,Reading park stabbings: UK Labour leader react...,runs its course but I will want to work with t...,0,runs its course but i will want to work with t...
3,3,lpYVmd8a1rY,lpYVmd8a1rY_3,Reading park stabbings: UK Labour leader react...,moment investigation has to take its course bu...,1,moment investigation has to take its course bu...
4,4,lpYVmd8a1rY,lpYVmd8a1rY_4,Reading park stabbings: UK Labour leader react...,our responders the NHS and the police yet agai...,0,our responders the nhs and the police yet agai...


# 3. Data Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['sentiment'], test_size=0.2, random_state=None)

# 4. Models

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [11]:
max_len = 115
vocab_size = len(tokenizer.word_index) + 1 

In [12]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(curve="ROC", summation_method="interpolation", name='auc')
]

## 4.1 LSTM

In [13]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=5000, output_dim=256, input_length=max_len)) 
lstm_model.add(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.001)))
lstm_model.add(Dropout(0.4))
lstm_model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.0001)
lstm_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=METRICS)

history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 106ms/step - accuracy: 0.6181 - auc: 0.5331 - loss: 0.9325 - precision: 0.5246 - recall: 0.1112 - val_accuracy: 0.5668 - val_auc: 0.5813 - val_loss: 0.9197 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.5874 - auc: 0.5285 - loss: 0.9055 - precision: 0.6000 - recall: 0.0034 - val_accuracy: 0.5668 - val_auc: 0.5870 - val_loss: 0.9019 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.6232 - auc: 0.5664 - loss: 0.8671 - precision: 0.7250 - recall: 0.0025 - val_accuracy: 0.5635 - val_auc: 0.6215 - val_loss: 0.8800 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.6078 - auc: 0.6670 - loss: 0.8434 - precision: 0.7308 - recall: 0.0089 - val_accuracy: 0.5570 - val_auc: 0.6610 - val_loss: 0.8376 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Ep

In [14]:
results = lstm_model.evaluate(X_test, y_test)

print('Results using LSTM:')
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')
print(f'Precision: {results[2]}')
print(f'Recall: {results[3]}')
print(f'AUC: {results[4]}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.7579 - auc: 0.8125 - loss: 0.6879 - precision: 0.7674 - recall: 0.6335
Results using LSTM:
Loss: 0.6628504991531372
Accuracy: 0.7687296271324158
Precision: 0.7719298005104065
Recall: 0.6616541147232056
AUC: 0.8304381370544434


## 4.2 BiLSTM

In [15]:
bilstm_model = Sequential()
bilstm_model.add(Embedding(input_dim=5000, output_dim=256, input_length=max_len))  
bilstm_model.add(Bidirectional(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.001)))) 
bilstm_model.add(Dropout(0.4))  
bilstm_model.add(Dense(1, activation='sigmoid')) 

optimizer = Adam(learning_rate=0.0001)
bilstm_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=METRICS)

history_bilstm = bilstm_model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 27s 72ms/step - accuracy: 0.6592 - auc: 0.6862 - loss: 1.1105 - precision: 0.7447 - recall: 0.2791 - val_accuracy: 0.5635 - val_auc: 0.7095 - val_loss: 0.9143 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - accuracy: 0.6521 - auc: 0.7619 - loss: 0.8257 - precision: 0.6358 - recall: 0.1940 - val_accuracy: 0.7003 - val_auc: 0.7919 - val_loss: 0.7349 - val_precision: 0.6814 - val_recall: 0.5789
Epoch 3/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step - accuracy: 0.7793 - auc: 0.8546 - loss: 0.6512 - precision: 0.7827 - recall: 0.6336 - val_accuracy: 0.7459 - val_auc: 0.8351 - val_loss: 0.6267 - val_precision: 0.6978 - val_recall: 0.7293
Epoch 4/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step - accuracy: 0.8698 - auc: 0.9186 - loss: 0.5020 - precision: 0.8674 - recall: 0.8155 - val_accuracy: 0.7655 - val_auc: 0.8537 - val_loss: 0.5676 - val_precision: 0.7652 - val_recall: 0.6617
Epoch 5/10
307/3

In [16]:
results = bilstm_model.evaluate(X_test, y_test)

print('Results using BiLSTM:')
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')
print(f'Precision: {results[2]}')
print(f'Recall: {results[3]}')
print(f'AUC: {results[4]}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7513 - auc: 0.8292 - loss: 0.6000 - precision: 0.7541 - recall: 0.6362
Results using BiLSTM:
Loss: 0.5675662159919739
Accuracy: 0.7654722929000854
Precision: 0.7652173638343811
Recall: 0.6616541147232056
AUC: 0.8537075519561768


## 4.3 CNN

In [17]:
cnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=256, input_length=max_len),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') 
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

history_cnn = cnn_model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.6769 - auc: 0.7473 - loss: 0.6393 - precision: 0.6405 - recall: 0.4686 - val_accuracy: 0.7036 - val_auc: 0.7886 - val_loss: 0.5809 - val_precision: 0.7019 - val_recall: 0.5489
Epoch 2/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.7905 - auc: 0.8615 - loss: 0.4606 - precision: 0.7272 - recall: 0.7359 - val_accuracy: 0.7427 - val_auc: 0.8573 - val_loss: 0.6801 - val_precision: 0.8462 - val_recall: 0.4962
Epoch 3/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9254 - auc: 0.9833 - loss: 0.1767 - precision: 0.9346 - recall: 0.8867 - val_accuracy: 0.7915 - val_auc: 0.8599 - val_loss: 0.6291 - val_precision: 0.7447 - val_recall: 0.7895


In [18]:
results = cnn_model.evaluate(X_test, y_test)

print('Results using CNN:')
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')
print(f'Precision: {results[2]}')
print(f'Recall: {results[3]}')
print(f'AUC: {results[4]}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7146 - auc: 0.7771 - loss: 0.5954 - precision: 0.7197 - recall: 0.5574
Results using CNN:
Loss: 0.5808533430099487
Accuracy: 0.7035830616950989
Precision: 0.7019230723381042
Recall: 0.548872172832489
AUC: 0.7885662317276001


## 4.4 CNN-RNN

In [19]:
cnn_rnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

cnn_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

history_cnn_rnn = cnn_rnn_model.fit(X_train, y_train, epochs=10, batch_size=4, 
                                     validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.6510 - auc: 0.7063 - loss: 0.6494 - precision: 0.5968 - recall: 0.4517 - val_accuracy: 0.5570 - val_auc: 0.5025 - val_loss: 0.6895 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.5889 - auc: 0.4959 - loss: 0.6828 - precision: 0.3201 - recall: 0.0362 - val_accuracy: 0.5668 - val_auc: 0.5232 - val_loss: 0.6840 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.6123 - auc: 0.5513 - loss: 0.6602 - precision: 0.1306 - recall: 0.0091 - val_accuracy: 0.7427 - val_auc: 0.8011 - val_loss: 0.5834 - val_precision: 0.7647 - val_recall: 0.5865


In [20]:
results = cnn_rnn_model.evaluate(X_test, y_test)

print('Results using CNN-RNN:')
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')
print(f'Precision: {results[2]}')
print(f'Recall: {results[3]}')
print(f'AUC: {results[4]}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5536 - auc: 0.5065 - loss: 0.6897 - precision: 0.0000e+00 - recall: 0.0000e+00
Results using CNN-RNN:
Loss: 0.6895025968551636
Accuracy: 0.5570032596588135
Precision: 0.0
Recall: 0.0
AUC: 0.5024846792221069


## 4.5 LSTM with self-attention

In [21]:
class SelfAttention(Layer):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def build(self, input_shape):
        self.Wq = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.Wk = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.Wv = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)

    def call(self, inputs):
        query = tf.matmul(inputs, self.Wq)
        key = tf.matmul(inputs, self.Wk)
        value = tf.matmul(inputs, self.Wv)
        
        scores = tf.matmul(query, tf.transpose(key, perm=[0, 2, 1])) / tf.sqrt(tf.cast(tf.shape(key)[-1], tf.float32))
        attention_weights = tf.nn.softmax(scores, axis=-1)
        
        output = tf.matmul(attention_weights, value)
        return output

In [22]:
lstm_sa_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    SelfAttention(), 
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

lstm_sa_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

history_lstm_sa = lstm_sa_model.fit(X_train, y_train, epochs=10, batch_size=4, 
                                     validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.6105 - auc: 0.6111 - loss: 0.6428 - precision: 0.5295 - recall: 0.1987 - val_accuracy: 0.6938 - val_auc: 0.7893 - val_loss: 0.5585 - val_precision: 0.6242 - val_recall: 0.7368
Epoch 2/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.8187 - auc: 0.8724 - loss: 0.4347 - precision: 0.8337 - recall: 0.6891 - val_accuracy: 0.7394 - val_auc: 0.8270 - val_loss: 0.5445 - val_precision: 0.7978 - val_recall: 0.5338
Epoch 3/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 15s 50ms/step - accuracy: 0.9385 - auc: 0.9788 - loss: 0.1828 - precision: 0.9456 - recall: 0.8957 - val_accuracy: 0.7427 - val_auc: 0.8429 - val_loss: 0.7596 - val_precision: 0.7812 - val_recall: 0.5639
Epoch 4/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.9810 - auc: 0.9938 - loss: 0.0792 - precision: 0.9804 - recall: 0.9693 - val_accuracy: 0.7818 - val_auc: 0.8419 - val_loss: 0.8463 - val_precision: 0.7391 - val_recall: 0.7669
Epoch 5/10
307/307 ━━━━━

In [23]:
results = lstm_sa_model.evaluate(X_test, y_test)

print('Results using LSTM with Self-Attention:')
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')
print(f'Precision: {results[2]}')
print(f'Recall: {results[3]}')
print(f'AUC: {results[4]}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7442 - auc: 0.8046 - loss: 0.5881 - precision: 0.8080 - recall: 0.5379
Results using LSTM with Self-Attention:
Loss: 0.544535219669342
Accuracy: 0.7394136786460876
Precision: 0.7977527976036072
Recall: 0.5338345766067505
AUC: 0.8269813060760498


## 4.6 RNN-Capsule

In [24]:
class CapsuleLayer(Layer):
    def __init__(self, num_capsules, dim_capsules, routing_iterations=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routing_iterations = routing_iterations

    def build(self, input_shape):
        
        self.W = self.add_weight(shape=[input_shape[-1], self.num_capsules * self.dim_capsules],
                                 initializer='glorot_uniform',
                                 trainable=True)
        
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        seq_len = tf.shape(inputs)[1]
        
        u_hat = tf.einsum('bij,jk->bik', inputs, self.W)
        u_hat = tf.reshape(u_hat, (batch_size, seq_len, self.num_capsules, self.dim_capsules))

        b = tf.zeros(shape=[batch_size, seq_len, self.num_capsules, 1])
        for i in range(self.routing_iterations):
            c = tf.nn.softmax(b, axis=2)
            s = tf.reduce_sum(c * u_hat, axis=1)
            v = self.squash(s)
            if i < self.routing_iterations - 1:
                b += tf.reduce_sum(u_hat * tf.expand_dims(v, 1), axis=-1, keepdims=True)

        return tf.reshape(v, [batch_size, self.num_capsules * self.dim_capsules])

    def squash(self, s):
        s_norm = tf.norm(s, axis=-1, keepdims=True)
        return (s_norm**2 / (1 + s_norm**2)) * (s / (s_norm + 1e-8))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules * self.dim_capsules)

In [25]:
rnn_capsule_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),  
    CapsuleLayer(num_capsules=10, dim_capsules=16),  
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')    
])

rnn_capsule_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

history_rnn_capsule = rnn_capsule_model.fit(X_train, y_train, epochs=10, batch_size=4, 
                                            validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 28s 73ms/step - accuracy: 0.6748 - auc: 0.7359 - loss: 0.6413 - precision: 0.6517 - recall: 0.4959 - val_accuracy: 0.7036 - val_auc: 0.7966 - val_loss: 0.5783 - val_precision: 0.8088 - val_recall: 0.4135
Epoch 2/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 70ms/step - accuracy: 0.8217 - auc: 0.8398 - loss: 0.4787 - precision: 0.8230 - recall: 0.6909 - val_accuracy: 0.7296 - val_auc: 0.8220 - val_loss: 0.5177 - val_precision: 0.8049 - val_recall: 0.4962
Epoch 3/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 22s 72ms/step - accuracy: 0.9156 - auc: 0.9403 - loss: 0.2741 - precision: 0.9455 - recall: 0.8349 - val_accuracy: 0.7818 - val_auc: 0.8228 - val_loss: 0.6167 - val_precision: 0.7797 - val_recall: 0.6917
Epoch 4/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.9638 - auc: 0.9745 - loss: 0.1516 - precision: 0.9679 - recall: 0.9249 - val_accuracy: 0.7557 - val_auc: 0.7954 - val_loss: 0.8212 - val_precision: 0.8222 - val_recall: 0.5564
Epoch 5/10
307/307 ━━━━━

In [26]:
results = rnn_capsule_model.evaluate(X_test, y_test)

print('Results using RNN-Capsule:')
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')
print(f'Precision: {results[2]}')
print(f'Recall: {results[3]}')
print(f'AUC: {results[4]}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7010 - auc: 0.7958 - loss: 0.5498 - precision: 0.7638 - recall: 0.4510
Results using RNN-Capsule:
Loss: 0.5177220106124878
Accuracy: 0.7296416759490967
Precision: 0.8048780560493469
Recall: 0.49624061584472656
AUC: 0.8219687342643738
